In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import copy
import tushare as ts
import math
import os
import sys

XIAOTIAN YANG
Quantitative Finance Internship Works
#2018
This file is merely for calculating the historical index weights for CSI300/CSI500/CSI50 approximately
It can' t fullfill the requirements for accurate uses.
In order to use this .py file, you have to have your data files prepared already with exact data format as required in the following.

All codes in the next cell are about how to read data

In [25]:
# CNE5S_100_Asset_Exposure = pd.read_table(read_path_input+'CNE5S_100_Asset_Exposure.'+target_date_rt, sep = '|', header=2)
# CNE5S_100_Asset_Exposure.rename(columns={'!Barrid':'Barrid'}, inplace = True)

# CNE5S_100_DlyFacRet = pd.read_table(read_path_input+'CNE5S_100_DlyFacRet.'+target_date_rt, sep = '|', header=2)
# CNE5S_100_DlyFacRet.rename(columns={'!Factor':'Factor'}, inplace = True)

# CNE5_Daily_Asset_Price = pd.read_table(read_path_input+'CNE5_Daily_Asset_Price.'+target_date_rt, sep = '|', header=1)
# CNE5_Daily_Asset_Price.rename(columns={'!Barrid':'Barrid', 'DlyReturn%':'DlyReturn'}, inplace = True)


# CNE5_100_Asset_DlySpecRet = pd.read_table(read_path_input+'CNE5_100_Asset_DlySpecRet.'+target_date_rt, sep = '|', header=2)
# CNE5_100_Asset_DlySpecRet.rename(columns={'!Barrid':'Barrid'}, inplace = True)

# CHN_LOCALID_Asset_ID = pd.read_csv(read_path_input+'CHN_LOCALID_Asset_ID.'+target_date_rt, sep = '|', header=1)
# CHN_LOCALID_Asset_ID.rename(columns={'!Barrid':'Barrid'}, inplace = True)
# CHN_LOCALID_Asset_ID = CHN_LOCALID_Asset_ID.dropna()
# CHN_LOCALID_Asset_ID = lastestBarrid(CHN_LOCALID_Asset_ID)

# CNE5S_100_Asset_Exposure = pd.read_table(read_path_input+'CNE5S_100_Asset_Exposure.'+target_date_rt, sep = '|', header=2)
# CNE5S_100_Asset_Exposure.rename(columns={'!Barrid':'Barrid'}, inplace = True)

In [140]:

class PassingError(Exception): 
        """Error encountered while passing arguments."""
        pass
    

def getwigt(alist):
    epylist = len(alist)*[None]
    for ii in range(len(alist)):
        epylist[ii] = alist[ii][1]
    return epylist


def getwigt_name(alist):
    epylist = len(alist)*[None]
    for ii in range(len(alist)):
        epylist[ii] = alist[ii][0]
    return epylist

def getwigt_localid(alist):
    epylist = len(alist)*[None]
    for ii in range(len(alist)):
        epylist[ii] = alist[ii][2]
    return epylist

def latest_id(local_id,iddf):
    dfa = iddf[iddf.AssetID.apply(lambda x: x == 'CN' + local_id[:6])]
    result = dfa[dfa.EndDate == max(dfa.EndDate)].values[0,0]
    return result

def Del_Narows(tgt_df, col_name):
    nalist = list(tgt_df.index[pd.isnull(tgt_df[col_name])].values)
    for na in nalist:
        tgt_df.drop(na,inplace = True)
    return tgt_df

def make_dict(keylist,vllist):
    emptydict = dict()
    if len(keylist) == len(vllist):
        for i in range(len(keylist)):
            emptydict[keylist[i]] = vllist[i]
        return emptydict
    else:
        print("Two lists are not the same long")
        
def lcl_bra(df1, df2):
    lcldict = []
    nanlist = []
    for index, row in df1.iterrows():
        try:
            barraid = latest_id(row[1], df2)
        except:
            print('\n' + '\033[1;31m' + row[1] + ' This is not included in barra' + '\033[0m')
            nanlist.append(row[0])
        else:
            lcldict.append([barraid,row[-1],row[1]])
    return lcldict, nanlist

def lcl_bra_init(df1, df2):
    lcldict = []
    nanlist = []
    for index, row in df1.iterrows():
        try:
            barraid = latest_id(row[0], df2)
        except:
            print('\n' + '\033[1;31m' + row[1] + ' This is not included in barra' + '\033[0m')
            nanlist.append(row[0])
        else:
            lcldict.append([barraid,row[-1],row[0]])
    return lcldict, nanlist

def Get_dfbylist_test(targetdf, key_nm, keylist, target_nm):
    emptylist = [0]*len(keylist)
    
    for i in range(len(keylist)):
        rowlist = [0,0]
        rowlist[0] = keylist[i]
        rowlist[1] = ((targetdf[targetdf[key_nm] == keylist[i]])[target_nm]).values[0]
        emptylist[i] = rowlist
        
    return emptylist

def Get_dfbylist(targetdf, key_nm, keylist, target_nm):
    emptylist = [0]*len(keylist)
    
    for i in range(len(keylist)):
        rowlist = [0]*2
        rowlist[0] = keylist[i]
        rowlist[1] = ((targetdf[targetdf[key_nm] == keylist[i]])[target_nm]).values[0]
        emptylist[i] = rowlist
    return emptylist

#This is a function use market value to calculate the change of index weights

def add_prep(df):
    preps = list(df.close)
    preps.insert(0,list(df.close)[0])
    df['pre_close'] = preps[:-1]
    return df

def date_cvt(astr):
    if len(astr) == 10:
        return astr[0:4]+astr[5:7]+astr[8:10]
    
    elif len(astr) == 8:
        return astr[0:4]+'-'+astr[4:6]+'-'+astr[6:8]
    
    else:
        raise PassingError('The passed argument is not as expected')

def Wigt_Cal_Mrt(local_idpair, wigts_crt_list, crt_index, tgt_index, crt_mrtvle_table, tgt_mrtvle_table, cal_direction):
    lgth_wigts = len(wigts_crt_list)
    empty_list = [None]*lgth_wigts
    temp_list = [None]*lgth_wigts
    
    if cal_direction == 'back':
        for index in range(lgth_wigts):
            temp_row = [None]*2
            wigt_pair = wigts_crt_list[index]
            idx_change = crt_index/tgt_index          
            crt_mrtvle = crt_mrtvle_table[crt_mrtvle_table['Barrid'] == wigt_pair[0]].Capt
            tgt_mrtvle = tgt_mrtvle_table[tgt_mrtvle_table['Barrid'] == latest_id(wigt_pair[2],local_idpair)].Capt
            idvl_mrtvle_change = (crt_mrtvle.values[0]/
                                      tgt_mrtvle.values[0])

            temp_row[0] = wigt_pair[2]
            temp_row[1] = (wigt_pair[1])*(idx_change)/(idvl_mrtvle_change)
            temp_list[index] = temp_row
        empty_list = [[x[0],(x[1]*100/sum(getwigt(temp_list)))] for x in temp_list]
            
    elif cal_direction == 'forward':
        
        for index in range(lgth_wigts):
            temp_row = [None]*2
            wigt_pair = wigts_crt_list[index]

            idx_change = tgt_index/crt_index
            crt_mrtvle = crt_mrtvle_table[crt_mrtvle_table['Barrid'] == wigt_pair[0]].Capt
            tgt_mrtvle = tgt_mrtvle_table[tgt_mrtvle_table['Barrid'] == latest_id(wigt_pair[2],local_idpair)].Capt
            idvl_mrtvle_change = (tgt_mrtvle.values[0]/
                                      crt_mrtvle.values[0])
            temp_row[0] = wigt_pair[2]
            temp_row[1] = wigt_pair[1]/(idx_change)*(idvl_mrtvle_change)
            temp_list[index] = temp_row
        empty_list = [[x[0],(x[1]*100/sum(getwigt(temp_list)))] for x in temp_list]

    else:
        raise PassingError('The passed argument is not as expected')
        
    return empty_list


def Weight_CalwithDate(start_triweight, start_date, trdys_list, index_table_input, calc_dicn, read_path_input):
    dly_rtdict = dict()
    for date in trdys_list:
        daily_return_table = Del_Narows(pd.read_table(read_path_input +'CNE5_Daily_Asset_Price.' +\
                                                              date, sep='|', header=1),'Capt')
        daily_return_table.rename(columns={'!Barrid': 'Barrid', 'DlyReturn%':'DlyReturn'}, inplace=True)
        dly_rtdict[date] = daily_return_table
    
    wigt_dict = dict()
    
    start_dtfmt2 = date_cvt(start_date)
    start_mrkt = dly_rtdict[start_date]

    for datea in (trdys_list):
        date_fmt2 = date_cvt(datea)
        
        localid_asset_id = pd.read_csv(read_path_input + 'CHN_LOCALID_Asset_ID.' + datea, sep='|', header=1)
        localid_asset_id.rename(columns={'!Barrid': 'Barrid'}, inplace=True)

        tgt_index_fc = index_table_input[index_table_input.date == date_fmt2].close.values[0]

        crt_index_fc = index_table_input[index_table_input.date == start_dtfmt2].close.values[0]
        
        temp_result = Wigt_Cal_Mrt(localid_asset_id, start_triweight, crt_index_fc, tgt_index_fc, start_mrkt, dly_rtdict[datea], calc_dicn)

        wigt_dict[datea] = temp_result
        
        print(datea+' calculated')
        
    return (wigt_dict,dly_rtdict)

In [144]:
def mainfunc(start_date, end_date, read_path_input,wight_file_path,index_type):
    
    start_date_dt = dt.datetime.strptime(start_date, '%Y%m%d')
    end_date_dt = dt.datetime.strptime(end_date, '%Y%m%d')

    if (start_date_dt - end_date_dt).days < 0:
        calc_dicn = 'forward'
        min_date = start_date
        max_date = end_date

    elif (start_date_dt - end_date_dt).days > 0:
        calc_dicn = 'back'
        max_date = start_date
        min_date = end_date

    else:
        raise PassingError('Start Date and End Date must be different')

    trade_days_index = ts.get_k_data(code = '000300', index = True,\
                                  start = date_cvt(min_date), end = date_cvt(max_date))

    trade_dates_list = list(trade_days_index.date.apply(lambda x : date_cvt(x)))
    print(trade_dates_list)
    chn_localid_asset_id = pd.read_csv(read_path_input + 'CHN_LOCALID_Asset_ID.' + start_date, sep='|', header=1)
    chn_localid_asset_id.rename(columns={'!Barrid': 'Barrid'}, inplace=True)
#read the initial weight file

    if index_type == 'hs300':
        idxPrc = add_prep(ts.get_k_data(code = '000300', index = True, start = '2014-12-01')).copy()[['date','close','pre_close']]
        indexWigt = pd.read_csv(wight_file_path + 'hs300_' + start_date+'.csv')
    elif index_type == 'zz500':
        idxPrc = add_prep(ts.get_k_data(code = '000905', index = True, start = '2014-12-01')).copy()[['date','close','pre_close']]
        indexWigt = pd.read_csv(wight_file_path + 'zz500_' + start_date+'.csv')
    elif index_type == 'sz50':
        idxPrc = add_prep(ts.get_k_data(code = '000016', index = True, start = '2014-12-01')).copy()[['date','close','pre_close']]
        indexWigt = pd.read_csv(wight_file_path + 'sz50_' + start_date+'.csv')
    else:
        raise PassingError('The passed argument is not as expected')
        
    indexWeight = indexWigt[['localid','weight']]
#     indexWeight = indexWigttp.rename(columns={'Date': 'date', 'Constituent_Code':'code','Constituent_Name':'name','Weight':'weight'})
    nmwigt_pair,nanlist = lcl_bra_init(Del_Narows(indexWeight,'weight'),chn_localid_asset_id)

    indexPrice = Del_Narows(idxPrc,'close')
    # indexPrice.drop(len(idxPrc)-1,inplace=True)
    indexPrice['close'] = idxPrc['close'].astype('float64')
    indexPrice['pre_close'] = idxPrc['pre_close'].astype('float64')

    index300_daily_weight,dly_rt = Weight_CalwithDate(nmwigt_pair, start_date, trade_dates_list,\
                                                  indexPrice, calc_dicn, read_path_input)
    return (index300_daily_weight,dly_rt)


def func_csv(index_daily_weight, pathin, indxmode):
    if indxmode == 'hs300':
        for exc in index_daily_weight:
            wgtdf = pd.DataFrame(data = np.array([getwigt_name(index_daily_weight[exc]),\
                                              getwigt(index_daily_weight[exc])]).T, columns=['localid','weight'])
            wgtdf.to_csv(pathin+'hs300w_'+exc+'.csv',index = None)

    if indxmode == 'zz500':
        for exc in index_daily_weight:
            wgtdf = pd.DataFrame(data = np.array([getwigt_name(index_daily_weight[exc]),\
                                              getwigt(index_daily_weight[exc])]).T, columns=['localid','weight'])
            wgtdf.to_csv(pathin+'zz500w_'+exc+'.csv',index = None)
            
    if indxmode == 'sz50':
        for exc in index_daily_weight:
            wgtdf = pd.DataFrame(data = np.array([getwigt_name(index_daily_weight[exc]),\
                                              getwigt(index_daily_weight[exc])]).T, columns=['localid','weight'])
            wgtdf.to_csv(pathin+'sz50w_'+exc+'.csv',index = None)

In [ ]:
# make a big dictionary containing all related dates in your calculation
date_dict = dict()
all_dates = ts.get_k_data(code = '000300', index = True,\
                            start = date_cvt('20171201'), end = date_cvt('20180631')).date
monthlist = []

# list all months and set up innitial varaibles
mmlist = ['1712','1801','1802','1803','1804','1805','1806']
headlist = ['hs300','sz50','zz500']
pathin = 'D:/datasets/'
pathout = 'out_weight/'
wigt_file_path = 'weight_in/'

# make integers for date_dict as keys and lists of dates within a certain month as values
for month in range(len(mmlist)):
    date_dict[month] = [date_cvt(x) for x in list(all_dates[all_dates.apply(lambda x: date_cvt(x)[:6] == '20'+mmlist[month])])]

for m in range(1,len(mmlist)):
    for h in headlist:
        startdt = date_dict[m-1][-1]
        enddt = date_dict[m][-2]
        indxmode = h
        print(startdt, enddt, pathin, mmlist[m], indxmode)
        index_daily_weight, dly_rt = mainfunc(startdt, enddt, pathin, wigt_file_path, indxmode)
        func_csv(index_daily_weight, pathout, indxmode)